# Basket option implementation with Bachelier model CV

In [70]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
import numpy as np
import pyfeng as pf

#import option_models as opt
import basket

In [72]:
# A trivial test case 1: 
# one asset have 100% weight (the others zero)
# the case should be equivalent to the BSM or Normal model price

spot = np.ones(4) * 100 
vol = np.ones(4) * 0.4
weights = np.array([1, 0, 0, 0])
divr = np.zeros(4)
intr = 0
cor_m = 0.5*np.identity(4) + 0.5 # 资产的相关系数矩阵
texp = 5
strike = 120

In [73]:
cor_m

array([[1. , 0.5, 0.5, 0.5],
       [0.5, 1. , 0.5, 0.5],
       [0.5, 0.5, 1. , 0.5],
       [0.5, 0.5, 0.5, 1. ]])

In [74]:
print(weights)

np.random.seed(123456)
price_basket = basket.basket_price_mc(strike, spot, vol*spot, weights, texp, cor_m, bsm=False)
print(price_basket)

[1 0 0 0]
26.570984651213593


In [75]:
# Compare the price to normal model formula

norm1 = pf.Norm(sigma=40)
price_norm = norm1.price(strike, spot[0], texp, cp=1)
print(price_basket, price_norm)

26.570984651213593 26.570845957870507


In [76]:
# A trivial test case 2
# all assets almost perfectly correlated:
# the case should be equivalent to the BSM or Normal model price

spot = np.ones(4) * 100
vol = np.ones(4) * 0.4
weights = np.ones(4) * 0.25
divr = np.zeros(4)
intr = 0
cor_m = 0.0001*np.identity(4) + 0.9999*np.ones((4,4))
texp = 5
strike = 120

print( cor_m )

np.random.seed(123456)
price_basket = basket.basket_price_mc(strike, spot, vol*spot, weights, texp, cor_m, bsm=False)
print(price_basket, price_norm)

[[1.     0.9999 0.9999 0.9999]
 [0.9999 1.     0.9999 0.9999]
 [0.9999 0.9999 1.     0.9999]
 [0.9999 0.9999 0.9999 1.    ]]
26.57211110181949 26.570845957870507


In [77]:
# A full test set for basket option with exact price

spot = np.ones(4) * 100
vol = np.ones(4) * 0.4
weights = np.ones(4) * 0.25
divr = np.zeros(4)
intr = 0
cor_m = 0.5*np.identity(4) + 0.5
texp = 5
strike = 100
price_exact = 28.0073695

In [78]:
weights, cor_m

(array([0.25, 0.25, 0.25, 0.25]),
 array([[1. , 0.5, 0.5, 0.5],
        [0.5, 1. , 0.5, 0.5],
        [0.5, 0.5, 1. , 0.5],
        [0.5, 0.5, 0.5, 1. ]]))

In [79]:
price_basket = basket.basket_price_mc(strike, spot, vol*spot, weights, texp, cor_m, bsm=False)
print(price_basket, price_exact)

28.22782882893394 28.0073695


# [To Do] Basket option implementation based on BSM model
## Write the similar test for BSM

In [80]:
price_basket = basket.basket_price_mc(strike, spot, vol, weights, texp, cor_m, bsm=True)
print(price_basket)

27.967309091912323


In [81]:
# A trivial test case 1: 
# one asset have 100% weight (the others zero)
# the case should be equivalent to the BSM or Normal model price

spot = np.ones(4) * 100
vol = np.ones(4) * 0.4
weights = np.array([1, 0, 0, 0])
divr = np.zeros(4)
intr = 0
cor_m = 0.5*np.identity(4) + 0.5
texp = 5
strike = 120

np.random.seed(123456)
price_basket = basket.basket_price_mc(strike, spot, vol, weights, texp, cor_m, bsm=True)

In [82]:
bsm1 = pf.Bsm(sigma=vol[0])
price_bsm = bsm1.price(strike, spot[0], texp, cp=1)
print(price_basket, price_bsm)

28.51439365199859 28.713486748445934


# Spread option implementation based on normal model

In [83]:
# A full test set for spread option

spot = np.array([100, 96])
vol = np.array([0.2, 0.1])
weights = np.array([1, -1])
divr = np.array([1, 1])*0.05
intr = 0.1
cor = 0.5
cor_m = np.array([[1, cor], [cor, 1]])
texp = 1
strike = 0
price_exact = 8.5132252

In [84]:
# MC price based on normal model
# make sure that the prices are similar

np.random.seed(123456)
price_spread = basket.basket_price_mc(strike, spot, vol*spot, weights, texp, cor_m, intr=intr, divr=divr, bsm=False)
print(price_spread, price_exact)

8.317680907159142 8.5132252


# Spread option implementation based on BSM model

In [85]:
# Once the implementation is finished the BSM model price should also work
price_spread = basket.basket_price_mc(
    strike, spot, vol, weights, texp, cor_m, intr=intr, divr=divr, bsm=True)
price_spread

8.530935560352313

In [86]:
# You also test Kirk's approximation
kirk = pf.BsmSpreadKirk(vol, cor=cor, divr=divr, intr=intr)
price_kirk = kirk.price(strike, spot, texp)
print(price_kirk, price_spread)

8.513225229545505 8.530935560352313


# [To Do] Complete the implementation of basket_price_norm_analytic
# Compare the MC stdev of BSM basket prices from with and without C

In [87]:
# The basket option example from above
spot = np.ones(4) * 100
vol = np.ones(4) * 0.4
weights = np.array([1, 1, 1, 1])/4
divr = np.zeros(4)
intr = 0
cor_m = 0.5*np.identity(4) + 0.5
texp = 5
strike = 120

In [88]:
### Make sure that the analytic normal price is correctly implemented

price_norm_analytic = basket.basket_price_norm_analytic(strike, spot, vol*spot, weights, texp, cor_m, intr=intr, divr=divr)
print(price_norm_analytic)

19.330395569726363


In [89]:
basket.basket_price_mc(strike, spot, vol, weights, texp, cor_m, intr=intr, divr=divr,bsm=True)

21.91271045394061

In [90]:
# Run below about 100 times and get the mean and stdev

### Returns 2 prices, without CV and with CV 
price_basket = basket.basket_price_mc_cv(strike, spot, vol, weights, texp, cor_m)
print(price_basket)

[21.62588623 21.66001515]


In [91]:
m_repeat = 100
value0 = np.zeros(m_repeat) # without CV
value1 = np.zeros(m_repeat) # with CV
for i in range(0,m_repeat):
    price_basket1 = basket.basket_price_mc_cv(strike, spot, vol, weights, texp, cor_m)
    value0[i] = price_basket1[0]
    value1[i] = price_basket1[1] 

In [95]:
print("without CV: [mean:{0},std:{1}]\nwith CV: [mean:{2}, std:{3}]".format(np.mean(value0), np.std(value0),np.mean(value1),np.std(value1)))

without CV: [mean:21.769116175187605,std:0.6205388835311394]
with CV: [mean:21.768757635091887, std:0.3429165776119549]
